# Nereus Diagnostics Demonstration

This notebook demonstrates the diagnostic functions in the `nereus` package for computing oceanographic metrics.

## Features covered:
- **Sea Ice Diagnostics**: ice_area, ice_volume, ice_extent
- **Ocean Diagnostics**: volume_mean, heat_content
- **Hovmoller Diagrams**: time-depth and time-latitude plots

## Setup and Data Loading

In [ ]:
import nereus as nr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

print(f"Nereus version: {nr.__version__}")

In [ ]:
# Load FESOM2 mesh and data
mesh = xr.open_dataset('/Users/nkolduno/PYTHON/DATA/CORE27_mesh/fesom.mesh.diag.nc')
data = xr.open_dataset('/Users/nkolduno/PYTHON/DATA/CORE27_data/temp.fesom.1958.nc')
data_u = xr.open_dataset('/Users/nkolduno/PYTHON/DATA/CORE27_data/u.fesom.1958.nc')
data_conc = xr.open_dataset('/Users/nkolduno/PYTHON/DATA/CORE27_data/a_ice.fesom.1958.nc')
data_thick = xr.open_dataset('/Users/nkolduno/PYTHON/DATA/CORE27_data/m_ice.fesom.1958.nc')

print(f"Mesh nodes: {len(mesh.lon)}")
print(f"Temperature shape: {data.temp.shape}")
print(f"Ice concentration shape: {data_conc.a_ice.shape}")
print(f"Ice thickness shape: {data_thick.m_ice.shape}")

In [ ]:
# Explore mesh variables
print("Mesh variables:")
for var in mesh.data_vars:
    print(f"  {var}: {mesh[var].dims} - shape {mesh[var].shape}")

In [ ]:
# Extract key mesh variables
lon = mesh.lon.values
lat = mesh.lat.values
area = mesh.nod_area.values  # Cell area in m^2

# Depth information (varies by mesh format)
if 'nz1' in data.coords:
    depth = data.nz1.values
elif 'depth' in mesh:
    depth = mesh.depth.values
else:
    depth = np.arange(data.temp.shape[1])  # placeholder

print(f"Coordinates: lon {lon.shape}, lat {lat.shape}")
print(f"Cell areas: {area.shape}, total area = {area.sum()/1e12:.2f} million km²")
print(f"Depth levels: {len(depth)} levels from {depth[0]:.1f}m to {depth[-1]:.1f}m")

---
## 1. Sea Ice Diagnostics

Nereus provides three sea ice diagnostic functions:
- `nr.ice_area()` - Total sea ice area (concentration-weighted)
- `nr.ice_volume()` - Total sea ice volume
- `nr.ice_extent()` - Sea ice extent (area above concentration threshold)

### 1.1 Explore Ice Data

In [ ]:
# Ice concentration (a_ice) - fraction from 0 to 1
sic = data_conc.a_ice  # Sea ice concentration
sit = data_thick.m_ice  # Sea ice thickness

print(f"Ice concentration: {sic.dims}")
print(f"Ice thickness: {sit.dims}")

# Check value ranges
print(f"\nConcentration range: {float(sic.min()):.3f} to {float(sic.max()):.3f}")
print(f"Thickness range: {float(sit.min()):.3f} to {float(sit.max()):.3f} m")

In [ ]:
# Visualize ice concentration for January
sic_jan = sic.isel(time=0).values

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Northern Hemisphere
fig, ax1, _ = nr.plot(
    sic_jan, lon, lat,
    projection="np",
    extent=(-180, 180, 50, 90),
    title="Arctic Ice Concentration (January 1958)",
    colorbar_label="Concentration",
    cmap="Blues",
    vmin=0, vmax=1
)

# Southern Hemisphere
fig, ax2, _ = nr.plot(
    sic_jan, lon, lat,
    projection="sp",
    extent=(-180, 180, -90, -50),
    title="Antarctic Ice Concentration (January 1958)",
    colorbar_label="Concentration",
    cmap="Blues",
    vmin=0, vmax=1
)

### 1.2 Ice Area

Sea ice area is computed as the sum of cell areas weighted by ice concentration:

$$\text{Ice Area} = \sum_i A_i \times C_i$$

where $A_i$ is the cell area and $C_i$ is the ice concentration.

In [ ]:
# Compute global ice area for all months
ice_area_global = []
for t in range(len(sic.time)):
    area_t = nr.ice_area(sic.isel(time=t).values, area)
    ice_area_global.append(area_t)

ice_area_global = np.array(ice_area_global)
print(f"Global ice area (January): {ice_area_global[0]/1e12:.2f} million km²")
print(f"Global ice area (September): {ice_area_global[8]/1e12:.2f} million km²")

In [ ]:
# Compute ice area by hemisphere using masks
nh_mask = lat > 0  # Northern Hemisphere
sh_mask = lat < 0  # Southern Hemisphere

ice_area_nh = []
ice_area_sh = []

for t in range(len(sic.time)):
    sic_t = sic.isel(time=t).values
    ice_area_nh.append(nr.ice_area(sic_t, area, mask=nh_mask))
    ice_area_sh.append(nr.ice_area(sic_t, area, mask=sh_mask))

ice_area_nh = np.array(ice_area_nh)
ice_area_sh = np.array(ice_area_sh)

print(f"\nNorthern Hemisphere:")
print(f"  Maximum (March): {ice_area_nh.max()/1e12:.2f} million km²")
print(f"  Minimum (September): {ice_area_nh.min()/1e12:.2f} million km²")

print(f"\nSouthern Hemisphere:")
print(f"  Maximum (September): {ice_area_sh.max()/1e12:.2f} million km²")
print(f"  Minimum (February): {ice_area_sh.min()/1e12:.2f} million km²")

In [ ]:
# Plot seasonal cycle of ice area
months = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(months, ice_area_nh / 1e12, 'b-o', label='Northern Hemisphere', linewidth=2)
ax.plot(months, ice_area_sh / 1e12, 'r-o', label='Southern Hemisphere', linewidth=2)
ax.set_xlabel('Month')
ax.set_ylabel('Ice Area (million km²)')
ax.set_title('Sea Ice Area Seasonal Cycle (1958)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()

### 1.3 Ice Extent

Sea ice extent is the total area of grid cells where ice concentration exceeds a threshold (typically 15%):

$$\text{Ice Extent} = \sum_i A_i \quad \text{where } C_i \geq 0.15$$

In [ ]:
# Compute ice extent (default threshold 15%)
ice_extent_nh = []
ice_extent_sh = []

for t in range(len(sic.time)):
    sic_t = sic.isel(time=t).values
    ice_extent_nh.append(nr.ice_extent(sic_t, area, mask=nh_mask))
    ice_extent_sh.append(nr.ice_extent(sic_t, area, mask=sh_mask))

ice_extent_nh = np.array(ice_extent_nh)
ice_extent_sh = np.array(ice_extent_sh)

print(f"Arctic ice extent (March): {ice_extent_nh[2]/1e12:.2f} million km²")
print(f"Arctic ice extent (September): {ice_extent_nh[8]/1e12:.2f} million km²")
print(f"Antarctic ice extent (September): {ice_extent_sh[8]/1e12:.2f} million km²")
print(f"Antarctic ice extent (February): {ice_extent_sh[1]/1e12:.2f} million km²")

In [ ]:
# Compare ice area vs ice extent
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Northern Hemisphere
axes[0].plot(months, ice_area_nh / 1e12, 'b-o', label='Ice Area', linewidth=2)
axes[0].plot(months, ice_extent_nh / 1e12, 'b--s', label='Ice Extent', linewidth=2)
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Area (million km²)')
axes[0].set_title('Arctic Ice Area vs Extent')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Southern Hemisphere
axes[1].plot(months, ice_area_sh / 1e12, 'r-o', label='Ice Area', linewidth=2)
axes[1].plot(months, ice_extent_sh / 1e12, 'r--s', label='Ice Extent', linewidth=2)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Area (million km²)')
axes[1].set_title('Antarctic Ice Area vs Extent')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()

In [ ]:
# Compare different concentration thresholds
thresholds = [0.10, 0.15, 0.30, 0.50]
sic_sept = sic.isel(time=8).values

print("September Arctic ice extent at different thresholds:")
for thresh in thresholds:
    extent = nr.ice_extent(sic_sept, area, threshold=thresh, mask=nh_mask)
    print(f"  {thresh*100:.0f}%: {extent/1e12:.2f} million km²")

### 1.4 Ice Volume

Sea ice volume combines thickness, area, and concentration:

$$\text{Ice Volume} = \sum_i h_i \times A_i \times C_i$$

where $h_i$ is the ice thickness.

In [ ]:
# Compute ice volume
ice_vol_nh = []
ice_vol_sh = []

for t in range(len(sic.time)):
    sic_t = sic.isel(time=t).values
    sit_t = sit.isel(time=t).values
    
    # Ice volume with concentration weighting
    ice_vol_nh.append(nr.ice_volume(sit_t, area, concentration=sic_t, mask=nh_mask))
    ice_vol_sh.append(nr.ice_volume(sit_t, area, concentration=sic_t, mask=sh_mask))

ice_vol_nh = np.array(ice_vol_nh)
ice_vol_sh = np.array(ice_vol_sh)

print(f"Arctic ice volume (April): {ice_vol_nh[3]/1e12:.1f} thousand km³")
print(f"Arctic ice volume (September): {ice_vol_nh[8]/1e12:.1f} thousand km³")
print(f"Antarctic ice volume (September): {ice_vol_sh[8]/1e12:.1f} thousand km³")

In [ ]:
# Plot ice volume seasonal cycle
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(months, ice_vol_nh / 1e12, 'b-o', label='Arctic', linewidth=2)
ax.plot(months, ice_vol_sh / 1e12, 'r-o', label='Antarctic', linewidth=2)
ax.set_xlabel('Month')
ax.set_ylabel('Ice Volume (thousand km³)')
ax.set_title('Sea Ice Volume Seasonal Cycle (1958)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()

---
## 2. Ocean Diagnostics

Nereus provides ocean diagnostic functions:
- `nr.volume_mean()` - Volume-weighted mean of a quantity
- `nr.heat_content()` - Ocean heat content calculation

### 2.1 Explore 3D Temperature Data

In [ ]:
# Temperature data structure
print(f"Temperature dimensions: {data.temp.dims}")
print(f"Temperature shape: {data.temp.shape}")
print(f"Depth levels: {len(depth)}")
print(f"\nFirst 10 depth levels (m): {depth[:10]}")

In [ ]:
# Compute layer thicknesses from depth levels
# For FESOM, depths represent level centers, so we compute thickness as the difference
if 'nz' in mesh.coords:
    # Use mesh-provided layer interfaces
    layer_interfaces = mesh.nz.values
    layer_thickness = np.diff(layer_interfaces)
else:
    # Estimate from depth centers
    layer_thickness = np.zeros(len(depth))
    layer_thickness[0] = depth[0] * 2  # First layer
    layer_thickness[1:] = np.diff(depth)
    # Approximate: use difference between consecutive depths

print(f"Layer thicknesses shape: {layer_thickness.shape}")
print(f"First 10 layer thicknesses (m): {layer_thickness[:10]}")
print(f"Total depth: {layer_thickness.sum():.1f} m")

### 2.2 Volume-Weighted Mean

The volume-weighted mean accounts for varying cell volumes:

$$\bar{T} = \frac{\sum_{i,k} T_{i,k} \times A_i \times \Delta z_k}{\sum_{i,k} A_i \times \Delta z_k}$$

In [ ]:
# Compute global mean temperature for January
temp_jan = data.temp.isel(time=0).values  # Shape: (nlevels, npoints)

# Full-depth mean
mean_temp_full = nr.volume_mean(temp_jan, area, layer_thickness)
print(f"Global mean temperature (full depth): {mean_temp_full:.2f}°C")

In [ ]:
# Mean temperature in different depth ranges
depth_ranges = [
    (0, 100, "Upper 100m"),
    (0, 500, "Upper 500m"),
    (0, 700, "Upper 700m (IPCC standard)"),
    (700, 2000, "700-2000m"),
    (2000, 6000, "Below 2000m"),
]

print("Mean temperature by depth range:")
for d_min, d_max, label in depth_ranges:
    mean_t = nr.volume_mean(
        temp_jan, area, layer_thickness, depth,
        depth_min=d_min, depth_max=d_max
    )
    print(f"  {label}: {mean_t:.2f}°C")

In [ ]:
# Compute monthly mean temperature time series
mean_temp_surface = []
mean_temp_700 = []
mean_temp_deep = []

for t in range(len(data.time)):
    temp_t = data.temp.isel(time=t).values
    
    mean_temp_surface.append(nr.volume_mean(
        temp_t, area, layer_thickness, depth, depth_max=100
    ))
    mean_temp_700.append(nr.volume_mean(
        temp_t, area, layer_thickness, depth, depth_max=700
    ))
    mean_temp_deep.append(nr.volume_mean(
        temp_t, area, layer_thickness, depth, depth_min=2000
    ))

mean_temp_surface = np.array(mean_temp_surface)
mean_temp_700 = np.array(mean_temp_700)
mean_temp_deep = np.array(mean_temp_deep)

In [ ]:
# Plot seasonal cycle of mean temperature
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(months, mean_temp_surface, 'r-o', label='Upper 100m', linewidth=2)
ax.plot(months, mean_temp_700, 'g-s', label='Upper 700m', linewidth=2)
ax.plot(months, mean_temp_deep, 'b-^', label='Below 2000m', linewidth=2)

ax.set_xlabel('Month')
ax.set_ylabel('Mean Temperature (°C)')
ax.set_title('Global Ocean Mean Temperature by Depth (1958)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()

In [ ]:
# Regional mean temperature (e.g., North Atlantic)
natl_mask = (lat > 20) & (lat < 70) & (lon > -80) & (lon < 0)

mean_temp_natl = nr.volume_mean(
    temp_jan, area, layer_thickness, depth,
    depth_max=700, mask=natl_mask
)
print(f"North Atlantic mean temperature (0-700m): {mean_temp_natl:.2f}°C")

# Southern Ocean
so_mask = lat < -60
mean_temp_so = nr.volume_mean(
    temp_jan, area, layer_thickness, depth,
    depth_max=700, mask=so_mask
)
print(f"Southern Ocean mean temperature (0-700m): {mean_temp_so:.2f}°C")

### 2.3 Ocean Heat Content

Ocean heat content (OHC) is calculated as:

$$\text{OHC} = \rho \, c_p \sum_{i,k} (T_{i,k} - T_{\text{ref}}) \times A_i \times \Delta z_k$$

where:
- $\rho$ = 1025 kg/m³ (seawater density)
- $c_p$ = 3985 J/(kg·K) (specific heat capacity)
- $T_{\text{ref}}$ = reference temperature (default 0°C)

In [ ]:
# Compute ocean heat content for January
ohc_full = nr.heat_content(temp_jan, area, layer_thickness)
print(f"Total OHC (full depth): {ohc_full:.3e} J")
print(f"                      = {ohc_full/1e21:.1f} ZJ (zettajoules)")

In [ ]:
# OHC by depth layer (IPCC standard layers)
ohc_0_700 = nr.heat_content(temp_jan, area, layer_thickness, depth, depth_max=700)
ohc_700_2000 = nr.heat_content(temp_jan, area, layer_thickness, depth, depth_min=700, depth_max=2000)
ohc_below_2000 = nr.heat_content(temp_jan, area, layer_thickness, depth, depth_min=2000)

print("Ocean Heat Content by depth layer:")
print(f"  0-700m:     {ohc_0_700/1e21:.1f} ZJ")
print(f"  700-2000m:  {ohc_700_2000/1e21:.1f} ZJ")
print(f"  Below 2000m: {ohc_below_2000/1e21:.1f} ZJ")
print(f"  Total:      {(ohc_0_700 + ohc_700_2000 + ohc_below_2000)/1e21:.1f} ZJ")

In [ ]:
# Monthly OHC time series
ohc_700_series = []
ohc_full_series = []

for t in range(len(data.time)):
    temp_t = data.temp.isel(time=t).values
    ohc_700_series.append(nr.heat_content(
        temp_t, area, layer_thickness, depth, depth_max=700
    ))
    ohc_full_series.append(nr.heat_content(
        temp_t, area, layer_thickness
    ))

ohc_700_series = np.array(ohc_700_series)
ohc_full_series = np.array(ohc_full_series)

In [ ]:
# Plot OHC seasonal cycle
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Absolute values
axes[0].plot(months, ohc_700_series / 1e21, 'b-o', label='0-700m', linewidth=2)
axes[0].set_xlabel('Month')
axes[0].set_ylabel('OHC (ZJ)')
axes[0].set_title('Ocean Heat Content (0-700m)')
axes[0].grid(True, alpha=0.3)

# Anomaly from annual mean
ohc_700_anom = ohc_700_series - ohc_700_series.mean()
axes[1].bar(months, ohc_700_anom / 1e21, color=['blue' if x < 0 else 'red' for x in ohc_700_anom])
axes[1].axhline(0, color='k', linewidth=0.5)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('OHC Anomaly (ZJ)')
axes[1].set_title('OHC Anomaly from Annual Mean (0-700m)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()

In [ ]:
# Effect of reference temperature
print("OHC with different reference temperatures:")
for t_ref in [0, 4, 10]:
    ohc = nr.heat_content(temp_jan, area, layer_thickness, depth, 
                         depth_max=700, reference_temp=t_ref)
    print(f"  T_ref = {t_ref}°C: {ohc/1e21:.1f} ZJ")

---
## 3. Hovmoller Diagrams

Hovmoller diagrams show the evolution of a quantity over time along one spatial dimension (depth or latitude).

- `nr.hovmoller()` - Compute Hovmoller data
- `nr.plot_hovmoller()` - Plot Hovmoller diagrams

### 3.1 Time-Depth Hovmoller

Shows how a quantity varies with depth over time (area-weighted mean at each depth level).

In [ ]:
# Get full year of temperature data
temp_all = data.temp.values  # Shape: (time, nlevels, npoints)
time_months = np.arange(1, 13)  # 1-12 for months

print(f"Temperature data shape: {temp_all.shape}")

# Compute time-depth Hovmoller (global mean)
t_out, z_out, hov_data = nr.hovmoller(
    temp_all, area,
    time=time_months,
    depth=depth,
    mode="depth"
)

print(f"Hovmoller output shape: {hov_data.shape}")

In [ ]:
# Plot time-depth Hovmoller
fig, ax = nr.plot_hovmoller(
    t_out, z_out, hov_data,
    mode="depth",
    cmap="RdYlBu_r",
    colorbar_label="Temperature (°C)",
    title="Global Mean Temperature (1958)",
    figsize=(12, 6)
)

ax.set_xlabel("Month")
ax.set_xticks(time_months)
ax.set_xticklabels(months)
plt.tight_layout()

In [ ]:
# Zoom in on upper ocean (0-500m)
upper_mask = depth <= 500
depth_upper = depth[upper_mask]
temp_upper = temp_all[:, upper_mask, :]

t_out, z_out, hov_upper = nr.hovmoller(
    temp_upper, area,
    time=time_months,
    depth=depth_upper,
    mode="depth"
)

fig, ax = nr.plot_hovmoller(
    t_out, z_out, hov_upper,
    mode="depth",
    cmap="RdYlBu_r",
    vmin=0, vmax=20,
    colorbar_label="Temperature (°C)",
    title="Upper Ocean Temperature (0-500m)",
    figsize=(12, 5)
)

ax.set_xlabel("Month")
ax.set_xticks(time_months)
ax.set_xticklabels(months)
plt.tight_layout()

In [ ]:
# Regional Hovmoller (North Atlantic)
natl_mask = (lat > 20) & (lat < 70) & (lon > -80) & (lon < 0)

t_out, z_out, hov_natl = nr.hovmoller(
    temp_upper, area,
    time=time_months,
    depth=depth_upper,
    mode="depth",
    mask=natl_mask
)

fig, ax = nr.plot_hovmoller(
    t_out, z_out, hov_natl,
    mode="depth",
    cmap="RdYlBu_r",
    vmin=5, vmax=25,
    colorbar_label="Temperature (°C)",
    title="North Atlantic Temperature (0-500m)",
    figsize=(12, 5)
)

ax.set_xlabel("Month")
ax.set_xticks(time_months)
ax.set_xticklabels(months)
plt.tight_layout()

### 3.2 Time-Latitude Hovmoller

Shows how surface quantities vary with latitude over time (zonal and depth mean).

In [ ]:
# Get SST data (surface temperature)
sst_all = data.temp.isel(nz1=0).values  # Shape: (time, npoints)

print(f"SST data shape: {sst_all.shape}")

# Compute time-latitude Hovmoller
t_out, lat_out, hov_lat = nr.hovmoller(
    sst_all, area,
    time=time_months,
    lat=lat,
    mode="latitude",
    lat_bins=np.arange(-90, 95, 5)  # 5-degree latitude bins
)

print(f"Latitude bins: {len(lat_out)}")
print(f"Hovmoller output shape: {hov_lat.shape}")

In [ ]:
# Plot time-latitude Hovmoller for SST
fig, ax = nr.plot_hovmoller(
    t_out, lat_out, hov_lat,
    mode="latitude",
    cmap="RdYlBu_r",
    vmin=-2, vmax=30,
    colorbar_label="SST (°C)",
    title="Sea Surface Temperature (1958)",
    figsize=(12, 6)
)

ax.set_xlabel("Month")
ax.set_xticks(time_months)
ax.set_xticklabels(months)
ax.axhline(0, color='k', linestyle='--', linewidth=0.5)  # Equator
plt.tight_layout()

In [ ]:
# Time-latitude Hovmoller for ice concentration
sic_all = sic.values  # Shape: (time, npoints)

t_out, lat_out, hov_ice = nr.hovmoller(
    sic_all, area,
    time=time_months,
    lat=lat,
    mode="latitude",
    lat_bins=np.arange(-90, 95, 2)  # Finer bins for ice
)

fig, ax = nr.plot_hovmoller(
    t_out, lat_out, hov_ice,
    mode="latitude",
    cmap="Blues",
    vmin=0, vmax=1,
    colorbar_label="Ice Concentration",
    title="Sea Ice Concentration (1958)",
    figsize=(12, 6)
)

ax.set_xlabel("Month")
ax.set_xticks(time_months)
ax.set_xticklabels(months)
ax.axhline(0, color='k', linestyle='--', linewidth=0.5)
plt.tight_layout()

In [ ]:
# Compute SST anomaly from zonal mean
# First compute annual mean at each latitude
hov_annual_mean = np.nanmean(hov_lat, axis=0)  # Mean over time
hov_anomaly = hov_lat - hov_annual_mean  # Anomaly from annual mean

fig, ax = nr.plot_hovmoller(
    t_out, lat_out, hov_anomaly,
    mode="latitude",
    cmap="RdBu_r",
    vmin=-5, vmax=5,
    colorbar_label="SST Anomaly (°C)",
    title="SST Anomaly from Annual Mean (1958)",
    figsize=(12, 6)
)

ax.set_xlabel("Month")
ax.set_xticks(time_months)
ax.set_xticklabels(months)
ax.axhline(0, color='k', linestyle='--', linewidth=0.5)
plt.tight_layout()

### 3.3 Custom Multi-Panel Hovmoller

In [ ]:
# Create multi-panel figure comparing different regions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Global time-depth
t_out, z_out, hov_global = nr.hovmoller(
    temp_upper, area, time=time_months, depth=depth_upper, mode="depth"
)
nr.plot_hovmoller(t_out, z_out, hov_global, mode="depth", ax=axes[0, 0],
                  cmap="RdYlBu_r", vmin=0, vmax=20,
                  title="Global", colorbar_label="T (°C)")

# North Atlantic time-depth
natl_mask = (lat > 20) & (lat < 70) & (lon > -80) & (lon < 0)
_, _, hov_natl = nr.hovmoller(
    temp_upper, area, time=time_months, depth=depth_upper, mode="depth", mask=natl_mask
)
nr.plot_hovmoller(t_out, z_out, hov_natl, mode="depth", ax=axes[0, 1],
                  cmap="RdYlBu_r", vmin=5, vmax=25,
                  title="North Atlantic", colorbar_label="T (°C)")

# Tropical Pacific time-depth
trop_pac_mask = (lat > -10) & (lat < 10) & ((lon > 120) | (lon < -80))
_, _, hov_tpac = nr.hovmoller(
    temp_upper, area, time=time_months, depth=depth_upper, mode="depth", mask=trop_pac_mask
)
nr.plot_hovmoller(t_out, z_out, hov_tpac, mode="depth", ax=axes[1, 0],
                  cmap="RdYlBu_r", vmin=10, vmax=30,
                  title="Tropical Pacific", colorbar_label="T (°C)")

# Southern Ocean time-depth
so_mask = lat < -60
_, _, hov_so = nr.hovmoller(
    temp_upper, area, time=time_months, depth=depth_upper, mode="depth", mask=so_mask
)
nr.plot_hovmoller(t_out, z_out, hov_so, mode="depth", ax=axes[1, 1],
                  cmap="RdYlBu_r", vmin=-2, vmax=5,
                  title="Southern Ocean", colorbar_label="T (°C)")

# Format x-axes
for ax in axes.flat:
    ax.set_xticks(time_months)
    ax.set_xticklabels(months)
    ax.set_xlabel("Month")

plt.suptitle("Regional Temperature Hovmoller Diagrams (0-500m, 1958)", fontsize=14, y=1.02)
plt.tight_layout()

---
## Summary

This notebook demonstrated the diagnostic functions in `nereus`:

### Sea Ice Diagnostics
- **`nr.ice_area(concentration, area, mask=None)`** - Total sea ice area (concentration-weighted)
- **`nr.ice_volume(thickness, area, concentration=None, mask=None)`** - Total sea ice volume
- **`nr.ice_extent(concentration, area, threshold=0.15, mask=None)`** - Sea ice extent

### Ocean Diagnostics
- **`nr.volume_mean(data, area, thickness, depth, depth_min, depth_max, mask)`** - Volume-weighted mean
- **`nr.heat_content(temp, area, thickness, depth, depth_min, depth_max, reference_temp, mask)`** - Ocean heat content

### Hovmoller Diagrams
- **`nr.hovmoller(data, area, time, depth/lat, mode)`** - Compute Hovmoller data
- **`nr.plot_hovmoller(time, y, data, mode)`** - Plot Hovmoller diagrams

### Key Features
- All functions support arbitrary masks for regional analysis
- Ocean diagnostics support depth range selection
- Works with both numpy arrays and xarray DataArrays
- Handles NaN values gracefully